# This should run when I use cite():

In [1]:
import bibtexparser
with open('/Users/kmt/Desktop/jupyter-references/readcube.bib') as f:
    ref_db = bibtexparser.load(f).entries
    # ref_db = {}
    # for entry in db.entries:
    #     ref_db[entry['ID']] = entry
ref_db[:1]

ModuleNotFoundError: No module named 'bibtexparser'

In [12]:
import json
with open('/Users/kmt/Desktop/jupyter-references/readcube.json') as f:
    ref_db = json.load(f)['items']
ref_db[:1]

[{'seq': 15227,
  'files': [],
  'article': {'isbn': None,
   'issn': '1424-8581',
   'year': 2007,
   'eisbn': None,
   'eissn': '1424-859X',
   'issue': '1-2',
   'title': 'Discontinuities and unsynapsed regions in meiotic chromosomes have a trans effect on meiotic recombination of some chromosomes in human males',
   'volume': '119',
   'authors': ['F Sun',
    'M Oliver-Bonet',
    'T Liehr',
    'H Starke',
    'E Ko',
    'A Rademaker',
    'R.H. Martin'],
   'chapter': None,
   'journal': 'Cytogenetic and Genome Research',
   'abstract': 'During meiosis, homologous chromosome pairing and synapsis are essential for subsequent meiotic recombination (crossing-over). Discontinuous regions (gaps) and unsynapsed regions (splits) were most frequently observed in the heterochromatic regions of bivalent synaptonemal complex (SC) 9, and we have previously demonstrated that gaps and splits significantly altered the distribution of MLH1 recombination foci on SC 9. Here, immunofluorescence t

In [24]:
import ipywidgets as widgets
import pandas as pd
from time import sleep

def search():
    w_list = []
    Authors = widgets.Text(description='Authors', continuous_update=False, layout=widgets.Layout(width='20%'))
    w_list.append(Authors)
    Year = widgets.Text(description='Year', continuous_update=False, layout=widgets.Layout(width='140px'))
    w_list.append(Year)
    Title = widgets.Text(description='Title', continuous_update=False, layout=widgets.Layout(width='20%'))
    w_list.append(Title)
    Journal = widgets.Text(description='Journal', continuous_update=False, layout=widgets.Layout(width='20%'))
    w_list.append(Journal)
    Select = widgets.Text(description='Select', continuous_update=False, layout=widgets.Layout(width='140px'))
    w_list.append(Select)

    def f(authors, year, title, journal, select):
        df = pd.DataFrame(dict(ID=['10110:/asdfk.sd2421']*5, Authors=[authors]*5, 
                               Year=[year]*5, Title=[title]*5, Journal=[journal]*5))
        df.set_index(df.index+1, inplace=True)
        if not select:
            with pd.option_context('display.max_colwidth', 100):
                left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
                left_aligned_df = left_aligned_df.set_table_styles(
                [dict(selector = 'th', props=[('text-align', 'left')])])

                display(left_aligned_df)
        else:
            # this closes all widgets except the output
            for w in w_list:
                w.close()
            # this closes output the outermost layer of the widget with the function output
            # out.close() 

            idx = list(map(int, select.split()))
            print('produce citation markdown for:', df.loc[idx])

            df.loc[idx].to_clipboard(index=False, header=False)
            print('selected bibtex entry copied to clipboard')

            return None

    out = widgets.interactive_output(f, {'authors': Authors, 'year': Year, 
                                         'title': Title, 'journal': Journal, 
                                         'select': Select})


    from IPython.display import clear_output
    print('Loading refernce database... (this is only done once)')
    sleep(1)
    clear_output()

    # widgets.VBox([widgets.VBox([Authors, Year, Title]), out])
    vb = widgets.HBox([Authors, Year, Title, Journal, Select])
    w_list.append(vb)
    vh = widgets.VBox([vb, out])
    
    return vh

search()

In [119]:
import bibtexparser
from collections import defaultdict
from thefuzz import fuzz, process
# https://app.datacamp.com/workspace/w/39daba8f-064f-4b3f-a1e6-7069fcb918b3
# https://github.com/seatgeek/thefuzz#simple-ratio

with open('readcube.bib') as bibtex_file:
    bib_database = bibtexparser.load(bibtex_file).entries
    
for i in range(len(bib_database)):
    if 'title' in  bib_database[i]:
        bib_database[i]['title'] = bib_database[i]['title'].replace('{', '').replace('}', '')
    
class left:
    def __rlshift__(self, df):
        "Left align columns of data frame: df << left()"
        left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
        left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector = 'th', props=[('text-align', 'left')])])
        display(left_aligned_df)  

In [168]:
author_strings = {} # [f'{i} {x}' for (i, x) in enumerate(author)]
title_strings = {} # [f'{i} {x}' for (i, x) in enumerate(title)]
journal_strings = {} # [f'{i} {x}' for (i, x) in enumerate(journal)]
year_strings = {} # [f'{i} {x}' for (i, x) in enumerate(year)]

year = '2016'

for i, entry in enumerate(bib_database):

    if not all(x in entry for x in ['author', 'year', 'title', 'journal']):
        continue
    
    if entry['year'] != year:
        continue
    
    author_list = entry['author'].split(' and ')
    author_list = [a.split(', ')[0] for a in author_list]
    
    if len(author_list) > 6:
        author_strings[i] = f"{i} {' '.join(author_list[:3] + author_list[-3:])}"
    else:
        author_strings[i] = f"{i} {' '.join(author_list[:6])}"

    title_strings[i] = f"{i} {entry['title'][1:-1]}"
    year_strings[i] = f"{i} {entry['year'][1:-1]}"
    journal_strings[i] = f"{i} {entry['journal'][1:-1]}"


author_hits = process.extractBests("Elyashiv", author_strings.values(), 
                     scorer=fuzz.partial_token_set_ratio, limit=3)
year_hits = process.extractBests('2016'.join(year), year_strings.values(), 
                     scorer=fuzz.partial_token_set_ratio, limit=3)
title_hits = process.extractBests('Selection'.join(names), title_strings.values(), 
                     scorer=fuzz.partial_token_set_ratio, limit=3)
journal_hits = process.extractBests('plos genetics'.join(names), journal_strings.values(), 
                     scorer=fuzz.partial_token_set_ratio, limit=3)

scores = defaultdict(int)
for hit, score in author_hits + year_hits + title_hits + journal_hits:
    i, string = hit.split(' ', 1)
    scores[int(i)] += score# / len(string)
    
    
_, db_idx = zip(*sorted([(v, k) for k, v in scores.items()], reverse=True))
result = [bib_database[i] for i in db_idx]
records = []

def format_row(r):
    author_list = r['author'].split(' and ')
    author_list = [a.split(', ')[0] for a in author_list]   
    if len(author_list) > 3:
        author = ', '.join(author_list[:2] + ['...'] +  author_list[-1:])
    else:
        author = ', '.join(author_list)
    if len(r['journal']) > 30:
        journal =  r['journal'][:30] + '...'
    else:
        journal =  r['journal']
    return [author, r['year'], r['title'], journal]

df = pd.DataFrame().from_records([format_row(r) for r in result],
                                 columns=['Author', 'Year', 'Title', 'Journal'])
df.index += 1
    
with pd.option_context('display.max_colwidth', 100):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector = 'th', props=[('text-align', 'left')])])
    display(left_aligned_df)

,Author,Year,Title,Journal
1,"Elyashiv, Sattath, ..., Sella",2016,A Genomic Map of the Effects of Linked Selection in Drosophila,PLOS Genetics
2,"Sankararaman, Mallick, ..., Reich",2016,The Combined Landscape of Denisovan and Neanderthal Ancestry in Present-Day Humans,Current Biology
3,"Vernot, Tucci, ..., Akey",2016,Excavating Neandertal and Denisovan DNA from the genomes of Melanesian individuals,Science
4,"Melkov, Baskar, ..., Abdu",2016,"A new mode of mitochondrial transport and polarized sorting regulated by Dynein, Milton and Miro",Development
5,"Griffiths, Jenkins, Lessard",2016,A coalescent dual process for a Wright-Fisher diffusion with recombination and its application to haplotype partitioning,Theoretical Population Biology
6,"Mendez, Poznik, ..., Bustamante",2016,The Divergence of Neandertal and Modern Human Y Chromosomes,The American Journal of Human ...
7,"Smagulova, Brick, ..., Petukhova",2016,The evolutionary turnover of recombination hot spots contributes to speciation in mice.,Genes \& development
8,"Whitlock, Peck, ..., Burch",2016,An Evolving Genetic Architecture Interacts with Hill-Robertson Interference to Determine the Benefit of Sex.,Genetics
9,"Mallick, Li, ..., Reich",2016,The Simons Genome Diversity Project: 300 genomes from 142 diverse populations,Nature
10,"Wright, Mudge, ..., Harrow",2016,Improving GENCODE reference gene annotation using a high-stringency proteogenomics workflow,Nature Communications


In [162]:
title_hits = process.extractBests("X", all_tokens, 
                     scorer=fuzz.partial_token_set_ratio, limit=3)
title_hits = process.extractBests("X", all_tokens, 
                     scorer=fuzz.partial_token_set_ratio, limit=3)


for hit, score in title_hits:
    search_id, search_string = hit.split(' ', 1)
    print(search_id)

year_hits = process.extractBests(' '.join(names), year, 
                     scorer=fuzz.partial_token_set_ratio, limit=3)
year_hits

Viuff,
Giovenino,
Skov,


[('2', 0), ('0', 0), ('1', 0)]

In [18]:
author_journal_hits[0], title_hits[0]

(('4 Skov, Laurits Macià, Moisès Coll Lucotte, Elise Anne Castellano, David Schierup, Mikkel Heide Munch, Kasper Cell Genomics',
  100),
 ('Viuff, Mette Skakkebæk, Anne Johannsen, Emma B. Trolle, Christian Just, Jesper Gravholt, Claus H. Genome Medicine X chromosome dosage and the genetic impact across human tissues 2023',
  100))

In [19]:
hits = process.extractBests("Skov  admixture", search_strings.keys(), 
                     scorer=fuzz.partial_token_set_ratio, limit=5)
keys, scores = zip(*hits)

tmpl = """{author}  
**{clean_title}**  
{journal} ({year})
---
"""
from IPython.display import Markdown, display

for key in keys:
    print(search_strings[key])
    display(Markdown(tmpl.format(**search_strings[key])))

{'pages': '100274', 'abstract': "{The X chromosome in non-African humans shows less diversity and less Neanderthal introgression than expected from neutral evolution. Analyzing 162 human male X chromosomes worldwide, we identified fourteen chromosomal regions where nearly identical haplotypes spanning several hundred kilobases are found at high frequencies in non-Africans. Genetic drift alone cannot explain the existence of these haplotypes, which must have been associated with strong positive selection in partial selective sweeps. Moreover, the swept haplotypes are entirely devoid of archaic ancestry as opposed to the non-swept haplotypes in the same genomic regions. The ancient Ust'-Ishim male dated at 45,000 before the present (BP)\xa0also carries the swept haplotypes, implying that selection on the haplotypes must have occurred between 45,000 and 55,000 years ago. Finally, we find that the chromosomal positions of sweeps overlap previously reported hotspots of selective\xa0sweeps i

Skov, Laurits and Macià, Moisès Coll and Lucotte, Elise Anne and Cavassim, Maria Izabel Alvez and Castellano, David and Schierup, Mikkel Heide and Munch, Kasper  
**Extraordinary selection on the human X chromosome associated with archaic admixture**  
Cell Genomics (2023)
---


{'abstract': '{Principal component analysis (PCA) is commonly used in genetics to infer and visualize population structure and admixture between populations. PCA is often interpreted in a way similar to inferred admixture proportions, where it is assumed that individuals belong to one of several possible populations or are admixed between these populations. We propose a new method to assess the statistical fit of PCA (interpreted as a model spanned by the top principal components) and to show that violations of the PCA assumptions affect the fit. Our method uses the chosen top principal components to predict the genotypes. By assessing the covariance (and the correlation) of the residuals (the differences between observed and predicted genotypes), we are able to detect violation of the model assumptions. Based on simulations and genome wide human data we show that our assessment of fit can be used to guide the interpretation of the data and to pinpoint individuals that are not well rep

Waaij, Jan van and Li, Song and Garcia-Erill, Genís and Albrechtsen, Anders and Wiuf, Carsten  
**Evaluation of population structure inferred by principal component analysis or the admixture model**  
arXiv (2023)
---


{'pages': '1--9', 'abstract': '{Despite major advances in genome technology and analysis, >50\\% of patients with a neurodevelopmental disorder (NDD) remain undiagnosed after extensive evaluation. A point in case is our clinically heterogeneous cohort of NDD patients that remained undiagnosed after FRAXA testing, chromosomal microarray analysis and trio exome sequencing (ES). In this study, we explored the frequency of non-random X chromosome inactivation (XCI) in the mothers of male patients and affected females, the rationale being that skewed XCI might be masking previously discarded genetic variants found on the X chromosome. A multiplex fluorescent PCR-based assay was used to analyse the pattern of XCI after digestion with HhaI methylation-sensitive restriction enzyme. In families with skewed XCI, we re-evaluated trio-based ES and identified pathogenic variants and a deletion on the X chromosome. Linkage analysis and RT-PCR were used to further study the inactive X chromosome alle

Giovenino, Chiara and Trajkova, Slavica and Pavinato, Lisa and Cardaropoli, Simona and Pullano, Verdiana and Ferrero, Enza and Sukarova-Angelovska, Elena and Carestiato, Silvia and Salmin, Paola and Rinninella, Antonina and Battaglia, Anthony and Bertoli, Luca and Fadda, Antonio and Palermo, Flavia and Carli, Diana and Mussa, Alessandro and Dimartino, Paola and Bruselles, Alessandro and Froukh, Tawfiq and Mandrile, Giorgia and Pasini, Barbara and Rubeis, Silvia De and Buxbaum, Joseph D. and Pippucci, Tommaso and Tartaglia, Marco and Rossato, Marzia and Delledonne, Massimo and Ferrero, Giovanni Battista and Brusco, Alfredo  
**Skewed X-chromosome inactivation in unsolved neurodevelopmental disease cases can guide re-evaluation For X-linked genes**  
European Journal of Human Genetics (2023)
---


{'volume': '613', 'number': '7943', 'pages': '308--316', 'abstract': '{The testis produces gametes through spermatogenesis and evolves rapidly at both the morphological and molecular level in mammals1–6, probably owing to the evolutionary pressure on males to be reproductively successful7. However, the molecular evolution of individual spermatogenic cell types across mammals remains largely uncharacterized. Here we report evolutionary analyses of single-nucleus transcriptome data for testes from 11 species that cover the three main mammalian lineages (eutherians, marsupials and monotremes) and birds (the evolutionary outgroup), and include seven primates. We find that the rapid evolution of the testis was driven by accelerated fixation rates of gene expression changes, amino acid substitutions and new genes in late spermatogenic stages, probably facilitated by reduced pleiotropic constraints, haploid selection and transcriptionally permissive chromatin. We identify temporal expression 

Murat, Florent and Mbengue, Noe and Winge, Sofia Boeg and Trefzer, Timo and Leushkin, Evgeny and Sepp, Mari and Cardoso-Moreira, Margarida and Schmidt, Julia and Schneider, Celine and Mößinger, Katharina and Brüning, Thoomke and Lamanna, Francesco and Belles, Meritxell Riera and Conrad, Christian and Kondova, Ivanela and Bontrop, Ronald and Behr, Rüdiger and Khaitovich, Philipp and Pääbo, Svante and Marques-Bonet, Tomas and Grützner, Frank and Almstrup, Kristian and Schierup, Mikkel Heide and Kaessmann, Henrik  
**The molecular evolution of spermatogenesis across mammals**  
Nature (2023)
---


{'volume': '15', 'number': '1', 'pages': '21', 'abstract': '{Sex chromosome aneuploidies (SCAs) give rise to a broad range of phenotypic traits and diseases. Previous studies based on peripheral blood samples have suggested the presence of ripple effects, caused by altered X chromosome number, affecting the methylome and transcriptome. Whether these alterations can be connected to disease-specific tissues, and thereby having clinical implication for the phenotype, remains to be elucidated. We performed a comprehensive analysis of X chromosome number on the transcriptome and methylome in blood, fat, and muscle tissue from individuals with 45,X, 46,XX, 46,XY, and 47,XXY. X chromosome number affected the transcriptome and methylome globally across all chromosomes in a tissue-specific manner. Furthermore, 45,X and 47,XXY demonstrated a divergent pattern of gene expression and methylation, with overall gene downregulation and hypomethylation in 45,X and gene upregulation and hypermethylatio

Viuff, Mette and Skakkebæk, Anne and Johannsen, Emma B. and Chang, Simon and Pedersen, Steen Bønlykke and Lauritsen, Katrine Meyer and Pedersen, Mette Glavind Bülow and Trolle, Christian and Just, Jesper and Gravholt, Claus H.  
**X chromosome dosage and the genetic impact across human tissues**  
Genome Medicine (2023)
---


In [6]:
%macro -q _cite https://gist.githubusercontent.com/kaspermunch/c4a64203651b054e18bae16792b828c6/raw/
%macro -q _incite https://gist.githubusercontent.com/kaspermunch/77f78d8d88a816a74b5415ac0640d2b7/raw/
%macro -q _reflist https://gist.githubusercontent.com/kaspermunch/977e903f6675b388998b1c3d367f5d55/raw/

[(Moretti 2017,](https://doi.org/10.1038/cdd.2017.32 "Moretti et al.
2017
SLY regulates genes involved in chromatin remodeling and interacts with TBL1XR1 during sperm differentiation
DOI:10.1038/cdd.2017.32") [Edme 2019,](https://doi.org/10.1111/jeb.13404 "Edme et al.
2019
Moderate heritability and low evolvability of sperm morphology in a species with high risk of sperm competition, the collared flycatcher Ficedula albicollis
DOI:10.1111/jeb.13404") [Julien 2012)](https://doi.org/10.1371/journal.pbio.1001328 "Julien et al.
2012
Mechanisms and Evolutionary Patterns of Mammalian and Avian Dosage Compensation
DOI:10.1371/journal.pbio.1001328")

## References

1. Edme et al., 2019, _Moderate heritability and low evolvability of sperm morphology in a species with high risk of sperm competition, the collared flycatcher Ficedula albicollis_, [DOI:10.1111/jeb.13404](https://doi.org/10.1111/jeb.13404)
2. Julien et al., 2012, _Mechanisms and Evolutionary Patterns of Mammalian and Avian Dosage Compensation_, [DOI:10.1371/journal.pbio.1001328](https://doi.org/10.1371/journal.pbio.1001328)
3. Moretti et al., 2017, _SLY regulates genes involved in chromatin remodeling and interacts with TBL1XR1 during sperm differentiation_, [DOI:10.1038/cdd.2017.32](https://doi.org/10.1038/cdd.2017.32)

In [22]:
# %load jupyter_cite.py
# def auth_year_cite():
#     match = re.search(r'@\w+{([^,]+).*year = {(\d+)}.*title = {{(.*)}}.*author = {([^,]+)',
#         clipboard.osx_clipboard_get(), re.DOTALL)
#     if match:
#         url, year, title, author = match.groups()
#         return f'[({author}, {year})]({url} "{title}")'
# if __name__ == "__main__":
#     link_text = auth_year_cite()
#     get_ipython().run_line_magic('load', '-n link_text')
    
# import re
# from IPython.lib import clipboard
# cb = clipboard.osx_clipboard_get()
# match = re.search(r'@\w+{([^,]+).*year = {(\d+)}.*title = {{(.*)}}.*author = {([^,]+)',
#     cb, re.DOTALL)
# if match:
#     url, year, title, author = match.groups()
#     link_text = f'[({author}, {year})](https://doi.org/{url} "{author} et al., {year},\n{title},\nDOI:{url}")'
#     get_ipython().run_line_magic('load', '-n link_text')    
#     # get_ipython().run_cell_magic('markdown', '', link_text)    
# else:
#     print('clipboard is not bibtex:', cb)

        
# import re
# from IPython.lib import clipboard

    
# from IPython.core.magic import Magics, magics_class, line_magic
# @magics_class
# class MyMagics(Magics):
#     @line_magic
#     def replace_content(self, line):
#         self.shell.set_next_input(line, replace=True)
# ip = get_ipython()
# ip.register_magics(MyMagics)
    
# cb = clipboard.osx_clipboard_get()
# bibtex_entries = re.findall(r'\@[^\@]+', cb)
# cite_list = []
# for i, entry in enumerate(bibtex_entries):
    
#     match = re.search(r'@\w+{([^,]+).*year = {(\d+)}.*title = {{(.*)}}.*author = {([^,]+)', entry, re.DOTALL)
#     if match:
#         url, year, title, author = match.groups()
#         if len(bibtex_entries) == 1:
#             cite_list.append(f'[({author} {year})](https://doi.org/{url} "{author} et al.\n{year}\n{title}\nDOI:{url}")')
#         else:
#             if i == 0:
#                 cite_list.append(f'[({author} {year},](https://doi.org/{url} "{author} et al.\n{year}\n{title}\nDOI:{url}")')
#             elif i+1 == len(bibtex_entries):
#                 cite_list.append(f'[{author} {year})](https://doi.org/{url} "{author} et al.\n{year}\n{title}\nDOI:{url}")')
#             else:
#                 cite_list.append(f'[{author} {year},](https://doi.org/{url} "{author} et al.\n{year}\n{title}\nDOI:{url}")')

# if cite_list:
#     content = ' '.join(cite_list)
#     # get_ipython().run_line_magic('load', '-n link_text')   
#     get_ipython().run_line_magic('replace_content', f"{content}") 
# else:
#     print('clipboard is not bibtex:', cb)
        
import os
import ipynbname
import json
import re
from IPython.lib import clipboard
from IPython.core.magic import Magics, magics_class, line_magic

@magics_class
class MyMagics(Magics):
    
    @line_magic
    def replace_content(self, line):
        self.shell.set_next_input(line, replace=True)
        
ip = get_ipython()
ip.register_magics(MyMagics)

def cite(cb=clipboard.osx_clipboard_get()):
    bibtex_entries = re.findall(r'\@[^\@]+', cb)
    cite_list = []
    for i, entry in enumerate(bibtex_entries):

        match = re.search(r'@\w+{([^,]+).*year = {(\d+)}.*title = {{(.*)}}.*author = {([^,]+)', entry, re.DOTALL)
        if match:
            url, year, title, author = match.groups()
            if len(bibtex_entries) == 1:
                cite_list.append(f'[({author} {year})](https://doi.org/{url} "{author} et al.\n{year}\n{title}\nDOI:{url}")')
            else:
                if i == 0:
                    cite_list.append(f'[({author} {year},](https://doi.org/{url} "{author} et al.\n{year}\n{title}\nDOI:{url}")')
                elif i+1 == len(bibtex_entries):
                    cite_list.append(f'[{author} {year})](https://doi.org/{url} "{author} et al.\n{year}\n{title}\nDOI:{url}")')
                else:
                    cite_list.append(f'[{author} {year},](https://doi.org/{url} "{author} et al.\n{year}\n{title}\nDOI:{url}")')
    content = ' '.join(cite_list)
    if content:
        get_ipython().run_line_magic('replace_content', f"{content}") 
    else:
        print('clipboard is not bibtex:', cb)
        
def incite(cb=clipboard.osx_clipboard_get()):
    bibtex_entries = re.findall(r'\@[^\@]+', cb)
    if len(bibtex_entries) > 1:
        print('clipboard has bibtex entries:', cb)
    else:
        match = re.search(r'@\w+{([^,]+).*year = {(\d+)}.*title = {{(.*)}}.*author = {([^,]+)',
            cb, re.DOTALL)
        if match:
            ref_list = {}
            url, year, title, author = match.groups()
            content = f'{author} et al. [({year})](https://doi.org/{url} "{author} et al.\n{year}\n{title}\nDOI:{url}")'
            get_ipython().run_line_magic('replace_content', f"{content}") 
        else:
            print('clipboard is not bibtex:', cb)
        
def reflist():
    regex = re.compile(r'\d+\s+"([^"]+)"')
    references = {}
    with open(os.path.abspath(ipynbname.name()+'.ipynb')) as f:
        notebook_json = json.load(f)
    for cell in notebook_json['cells']:
        if cell['cell_type'] == 'markdown':
            source = ''.join(cell['source'])
            for ref in regex.findall(source):
                author, year, title, doi = ref.split('\n')
                references[ref] = dict(author=author.strip(),
                                       year=year.strip(), 
                                       title=title.strip(),
                                       doi=doi.strip())
    lst = []
    for i, (key, ref) in enumerate(sorted(references.items())):
        lst.append(f"{i+1}. {ref['author']}, {ref['year']}, _{ref['title']}_, [{ref['doi']}](https://doi.org/{ref['doi'].replace('DOI:', '')})")
    content = "\n".join(lst)
    content = '## References\n\n' + content
    get_ipython().run_line_magic('replace_content', f"{content}") 
            
    

[(Charron 2019)](https://doi.org/10.1371/journal.pgen.1007964 "Charron et al.
2019
Two isoforms of the RAC-specific guanine nucleotide exchange factor TIAM2 act oppositely on transmission ratio distortion by the mouse t-haplotype
DOI:10.1371/journal.pgen.1007964")

## References

1. Aktary et al., 2021, _A role for Dynlt3 in melanosome movement, distribution, acidity and transfer_, [DOI:10.1038/s42003-021-01917-5](https://doi.org/10.1038/s42003-021-01917-5)
2. Edme et al., 2019, _Moderate heritability and low evolvability of sperm morphology in a species with high risk of sperm competition, the collared flycatcher Ficedula albicollis_, [DOI:10.1111/jeb.13404](https://doi.org/10.1111/jeb.13404)
3. Julien et al., 2012, _Mechanisms and Evolutionary Patterns of Mammalian and Avian Dosage Compensation_, [DOI:10.1371/journal.pbio.1001328](https://doi.org/10.1371/journal.pbio.1001328)
4. Moretti et al., 2017, _SLY regulates genes involved in chromatin remodeling and interacts with TBL1XR1 during sperm differentiation_, [DOI:10.1038/cdd.2017.32](https://doi.org/10.1038/cdd.2017.32)

[(Aktary 2021)](https://doi.org/10.1038/s42003-021-01917-5 "Aktary et al.
2021
A role for Dynlt3 in melanosome movement, distribution, acidity and transfer
DOI:10.1038/s42003-021-01917-5")

In [14]:
# import re
# from IPython.lib import clipboard

# from IPython.core.magic import Magics, magics_class, line_magic
# @magics_class
# class MyMagics(Magics):
#     @line_magic
#     def replace_content(self, line):
#         self.shell.set_next_input(line, replace=True)
# ip = get_ipython()
# ip.register_magics(MyMagics)

# def _cite(cb=clipboard.osx_clipboard_get()):
#     bibtex_entries = re.findall(r'\@[^\@]+', cb)
#     cite_list = []
#     for i, entry in enumerate(bibtex_entries):

#         match = re.search(r'@\w+{([^,]+).*year = {(\d+)}.*title = {{(.*)}}.*author = {([^,]+)', entry, re.DOTALL)
#         if match:
#             url, year, title, author = match.groups()
#             if len(bibtex_entries) == 1:
#                 cite_list.append(f'[({author} {year})](https://doi.org/{url} "{author} et al.\n{year}\n{title}\nDOI:{url}")')
#             else:
#                 if i == 0:
#                     cite_list.append(f'[({author} {year},](https://doi.org/{url} "{author} et al.\n{year}\n{title}\nDOI:{url}")')
#                 elif i+1 == len(bibtex_entries):
#                     cite_list.append(f'[{author} {year})](https://doi.org/{url} "{author} et al.\n{year}\n{title}\nDOI:{url}")')
#                 else:
#                     cite_list.append(f'[{author} {year},](https://doi.org/{url} "{author} et al.\n{year}\n{title}\nDOI:{url}")')
#     content = ' '.join(cite_list)
#     if content:
#         get_ipython().run_line_magic('replace_content', f"{content}") 
#     else:
#         print('clipboard is not bibtex:', cb)

# _cite()



In [12]:
_cite("""
@article{10.1038/cdd.2017.32, 
year = {2017}, 
title = {{SLY regulates genes involved in chromatin remodeling and interacts with TBL1XR1 during sperm differentiation}}, 
author = {Moretti, Charlotte and Serrentino, Maria-Elisabetta and Ialy-Radio, Côme and Delessard, Marion and Soboleva, Tatiana A and Tores, Frederic and Leduc, Marjorie and Nitschké, Patrick and Drevet, Joel R and Tremethick, David J and Vaiman, Daniel and Kocer, Ayhan and Cocquet, Julie}, 
journal = {Cell Death \& Differentiation}, 
issn = {1350-9047}, 
doi = {10.1038/cdd.2017.32}, 
pmid = {28475176}, 
pmcid = {PMC5442469}, 
abstract = {{Sperm differentiation requires unique transcriptional regulation and chromatin remodeling after meiosis to ensure proper compaction and protection of the paternal genome. Abnormal sperm chromatin remodeling can induce sperm DNA damage, embryo lethality and male infertility, yet, little is known about the factors which regulate this process. Deficiency in Sly, a mouse Y chromosome-encoded gene expressed only in postmeiotic male germ cells, has been shown to result in the deregulation of hundreds of sex chromosome-encoded genes associated with multiple sperm differentiation defects and subsequent male infertility. The underlying mechanism remained, to date, unknown. Here, we show that SLY binds to the promoter of sex chromosome-encoded and autosomal genes highly expressed postmeiotically and involved in chromatin regulation. Specifically, we demonstrate that Sly knockdown directly induces the deregulation of sex chromosome-encoded H2A variants and of the H3K79 methyltransferase DOT1L. The modifications prompted by loss of Sly alter the postmeiotic chromatin structure and ultimately result in abnormal sperm chromatin remodeling with negative consequences on the sperm genome integrity. Altogether our results show that SLY is a regulator of sperm chromatin remodeling. Finally we identified that SMRT/N-CoR repressor complex is involved in gene regulation during sperm differentiation since members of this complex, in particular TBL1XR1, interact with SLY in postmeiotic male germ cells.}}, 
pages = {1029--1044}, 
number = {6}, 
volume = {24}, 
keywords = {}
}""")

In [20]:
_cite()